# K-means (MADlib v1.11+)
Demonstrates k-means including new array input in 1.10 and new array unnest function in 1.11.

In [30]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [31]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [32]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.11-dev, git revision: rel/v1.10.0-26-ga3d54be, cmake configuration time: Thu Apr 27 01:01:30 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


## 1. Prepare some input data:

In [35]:
%%sql
DROP TABLE IF EXISTS km_sample;

CREATE TABLE km_sample(pid int, points double precision[]);

INSERT INTO km_sample VALUES
(1,  '{14.23, 1.71, 2.43, 15.6, 127, 2.8, 3.0600, 0.2800, 2.29, 5.64, 1.04, 3.92, 1065}'),
(2,  '{13.2, 1.78, 2.14, 11.2, 1, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050}'),
(3,  '{13.16, 2.36,  2.67, 18.6, 101, 2.8,  3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185}'),
(4,  '{14.37, 1.95, 2.5, 16.8, 113, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480}'),
(5,  '{13.24, 2.59, 2.87, 21, 118, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735}'),
(6,  '{14.2, 1.76, 2.45, 15.2, 112, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450}'),
(7,  '{14.39, 1.87, 2.45, 14.6, 96, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290}'),
(8,  '{14.06, 2.15, 2.61, 17.6, 121, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295}'),
(9,  '{14.83, 1.64, 2.17, 14, 97, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045}'),
(10, '{13.86, 1.35, 2.27, 16, 98, 2.98, 3.15, 0.22, 1.8500, 7.2199, 1.01, 3.55, 1045}');

SELECT * FROM km_sample ORDER BY pid;

Done.
Done.
10 rows affected.
10 rows affected.


pid,points
1,"[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]"
2,"[13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0]"
3,"[13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185.0]"
4,"[14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0]"
5,"[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0]"
6,"[14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0]"
7,"[14.39, 1.87, 2.45, 14.6, 96.0, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290.0]"
8,"[14.06, 2.15, 2.61, 17.6, 121.0, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295.0]"
9,"[14.83, 1.64, 2.17, 14.0, 97.0, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045.0]"
10,"[13.86, 1.35, 2.27, 16.0, 98.0, 2.98, 3.15, 0.22, 1.85, 7.2199, 1.01, 3.55, 1045.0]"


## 2. Run k-means clustering using kmeans++ with centroid seeding:

In [23]:
%%sql
DROP TABLE IF EXISTS km_result;

-- Run kmeans algorithm
CREATE TABLE km_result AS
SELECT * FROM madlib.kmeanspp( 'km_sample',   -- Table of source data
                               'points',      -- Column containing point co-ordinates 
                               2,             -- Number of centroids to calculate
                               'madlib.squared_dist_norm2',   -- Distance function
                               'madlib.avg',  -- Aggregate function
                               20,            -- Number of iterations
                               0.001          -- Fraction of centroids reassigned to keep iterating 
                             );

SELECT * FROM km_result;

Done.
1 rows affected.
1 rows affected.


centroids,cluster_variance,objective_fn,frac_reassigned,num_iterations
"[[13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0], [14.036, 2.018, 2.536, 16.56, 108.6, 3.004, 3.03, 0.298, 2.038, 6.10598, 1.004, 3.326, 1340.0]]","[90512.324426408, 60672.638245208]",151184.962672,0.0,2


## 3. Calculate the simplified silhouette coefficient:

In [24]:
%%sql
SELECT * FROM madlib.simple_silhouette( 'km_sample',          -- Input points table
                                        'points',             -- Column containing points
                                        (SELECT centroids FROM km_result),  -- Centroids
                                        'madlib.dist_norm2'   -- Distance function
                                      );

1 rows affected.


simple_silhouette
0.689788048829


## 4. Find the cluster assignment for each point:

In [37]:
%%sql
SELECT data.*,  (madlib.closest_column(centroids, points)).column_id as cluster_id
FROM km_sample as data, km_result
ORDER BY data.pid;

10 rows affected.


pid,points,cluster_id
1,"[14.23, 1.71, 2.43, 15.6, 127.0, 2.8, 3.06, 0.28, 2.29, 5.64, 1.04, 3.92, 1065.0]",0
2,"[13.2, 1.78, 2.14, 11.2, 1.0, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050.0]",0
3,"[13.16, 2.36, 2.67, 18.6, 101.0, 2.8, 3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185.0]",1
4,"[14.37, 1.95, 2.5, 16.8, 113.0, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480.0]",1
5,"[13.24, 2.59, 2.87, 21.0, 118.0, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735.0]",0
6,"[14.2, 1.76, 2.45, 15.2, 112.0, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450.0]",1
7,"[14.39, 1.87, 2.45, 14.6, 96.0, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290.0]",1
8,"[14.06, 2.15, 2.61, 17.6, 121.0, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295.0]",1
9,"[14.83, 1.64, 2.17, 14.0, 97.0, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045.0]",0
10,"[13.86, 1.35, 2.27, 16.0, 98.0, 2.98, 3.15, 0.22, 1.85, 7.2199, 1.01, 3.55, 1045.0]",0


## 5. Unnest the cluster centroids 2-D array to get a set of 1-D centroid arrays

In [40]:
%%sql
DROP TABLE IF EXISTS km_centroids_unnest;

-- Run unnest function
CREATE TABLE km_centroids_unnest AS
SELECT (madlib.array_unnest_2d_to_1d(centroids)).*
FROM km_result;

SELECT * FROM km_centroids_unnest ORDER BY 1;

Done.
2 rows affected.
2 rows affected.


unnest_row_id,unnest_result
1,"[13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0]"
2,"[14.036, 2.018, 2.536, 16.56, 108.6, 3.004, 3.03, 0.298, 2.038, 6.10598, 1.004, 3.326, 1340.0]"


Note that the ID column returned by array_unnest_2d_to_1d() is not the same as the cluster ID assigned by k-means. See below to display the cluster IDs.

## 6.  Display cluster ID
Create cluster IDs for 1-D centroid arrays so that cluster ID for any centroid can be matched to the cluster assignment for the data points:

In [39]:
%%sql
SELECT cent.*,  (madlib.closest_column(centroids, unnest_result)).column_id as cluster_id
FROM km_centroids_unnest as cent, km_result
ORDER BY cent.unnest_row_id;

2 rows affected.


unnest_row_id,unnest_result,cluster_id
1,"[13.872, 1.814, 2.376, 15.56, 88.2, 2.806, 2.928, 0.288, 1.844, 5.35198, 1.044, 3.348, 988.0]",0
2,"[14.036, 2.018, 2.536, 16.56, 108.6, 3.004, 3.03, 0.298, 2.038, 6.10598, 1.004, 3.326, 1340.0]",1


## 7. Array input

In [64]:
%%sql
DROP TABLE IF EXISTS km_arrayin CASCADE;

CREATE TABLE km_arrayin(pid int, 
                        p1 float, 
                        p2 float, 
                        p3 float,
                        p4 float, 
                        p5 float, 
                        p6 float,
                        p7 float, 
                        p8 float, 
                        p9 float,
                        p10 float, 
                        p11 float, 
                        p12 float,
                        p13 float);

INSERT INTO km_arrayin VALUES
(1,  14.23, 1.71, 2.43, 15.6, 127, 2.8, 3.0600, 0.2800, 2.29, 5.64, 1.04, 3.92, 1065),
(2,  13.2, 1.78, 2.14, 11.2, 1, 2.65, 2.76, 0.26, 1.28, 4.38, 1.05, 3.49, 1050),
(3,  13.16, 2.36,  2.67, 18.6, 101, 2.8,  3.24, 0.3, 2.81, 5.6799, 1.03, 3.17, 1185),
(4,  14.37, 1.95, 2.5, 16.8, 113, 3.85, 3.49, 0.24, 2.18, 7.8, 0.86, 3.45, 1480),
(5,  13.24, 2.59, 2.87, 21, 118, 2.8, 2.69, 0.39, 1.82, 4.32, 1.04, 2.93, 735),
(6,  14.2, 1.76, 2.45, 15.2, 112, 3.27, 3.39, 0.34, 1.97, 6.75, 1.05, 2.85, 1450),
(7,  14.39, 1.87, 2.45, 14.6, 96, 2.5, 2.52, 0.3, 1.98, 5.25, 1.02, 3.58, 1290),
(8,  14.06, 2.15, 2.61, 17.6, 121, 2.6, 2.51, 0.31, 1.25, 5.05, 1.06, 3.58, 1295),
(9,  14.83, 1.64, 2.17, 14, 97, 2.8, 2.98, 0.29, 1.98, 5.2, 1.08, 2.85, 1045),
(10, 13.86, 1.35, 2.27, 16, 98, 2.98, 3.15, 0.22, 1.8500, 7.2199, 1.01, 3.55, 1045);

SELECT * FROM km_arrayin ORDER BY pid;

Done.
Done.
10 rows affected.
10 rows affected.


pid,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13
1,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
2,13.2,1.78,2.14,11.2,1.0,2.65,2.76,0.26,1.28,4.38,1.05,3.49,1050.0
3,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.6799,1.03,3.17,1185.0
4,14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0
5,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0
6,14.2,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0
7,14.39,1.87,2.45,14.6,96.0,2.5,2.52,0.3,1.98,5.25,1.02,3.58,1290.0
8,14.06,2.15,2.61,17.6,121.0,2.6,2.51,0.31,1.25,5.05,1.06,3.58,1295.0
9,14.83,1.64,2.17,14.0,97.0,2.8,2.98,0.29,1.98,5.2,1.08,2.85,1045.0
10,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.2199,1.01,3.55,1045.0


In [67]:
%%sql
DROP TABLE IF EXISTS km_result;

-- Run kmeans algorithm
CREATE TABLE km_result AS
SELECT * FROM madlib.kmeans_random('km_arrayin', 
                                'ARRAY[p1, p2, p3, p4, p5, p6, 
                                      p7, p8, p9, p10, p11, p12, p13]', 
                                2,
                                'madlib.squared_dist_norm2',
                                'madlib.avg', 
                                20, 
                                0.001);

-- Get point assignment
SELECT data.*,  (madlib.closest_column(centroids, 
                                       ARRAY[p1, p2, p3, p4, p5, p6, 
                                      p7, p8, p9, p10, p11, p12, p13])).column_id as cluster_id
FROM km_arrayin as data, km_result
ORDER BY data.pid;

Done.
1 rows affected.
10 rows affected.


pid,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,cluster_id
1,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
2,13.2,1.78,2.14,11.2,1.0,2.65,2.76,0.26,1.28,4.38,1.05,3.49,1050.0,1
3,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.6799,1.03,3.17,1185.0,1
4,14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0,0
5,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1
6,14.2,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0,0
7,14.39,1.87,2.45,14.6,96.0,2.5,2.52,0.3,1.98,5.25,1.02,3.58,1290.0,0
8,14.06,2.15,2.61,17.6,121.0,2.6,2.51,0.31,1.25,5.05,1.06,3.58,1295.0,0
9,14.83,1.64,2.17,14.0,97.0,2.8,2.98,0.29,1.98,5.2,1.08,2.85,1045.0,1
10,13.86,1.35,2.27,16.0,98.0,2.98,3.15,0.22,1.85,7.2199,1.01,3.55,1045.0,1
